In [0]:
%run "../Configuration"

In [0]:
# Define the metadata widget:
dbutils.widgets.text("entityName", "")
dbutils.widgets.text("sourceSystem", "")

# Save parameters to variables:
entityName =       dbutils.widgets.get('entityName')
sourceSystemName = dbutils.widgets.get('sourceSystem')

In [0]:
# Define query to extract entityNames from sourceSystem table:
query = f"""
  SELECT entityNames
  FROM   sourceSystem
  WHERE  sourceSystemName = 'AdventureWorks'
  """

# Call query_entityNames function and load json:
entity_names = json.loads(query_entityNames(query))

In [0]:
print(entity_names)

In [0]:
# For each entity, create a dataFrame:
for entity in entity_names:
   # Read from SILVER path in metadata database:
   globals()[f"df_{entity}"] = spark.read.format('delta').load(SILVER + entity_silver(entity))

In [0]:
# Iterate through entities to create temp views:
for entity in entity_names:
    # Create or replace tempView with nomenclature tv_design, tv_counterparty etc:
    globals()[f'df_{entity}'].createOrReplaceTempView(f"tv_{entity}")

In [0]:
if entityName == 'customer_AW':
    query = """
    SELECT 
          customer_id
        , SPLIT(CustomerName, ' ')[0] AS first_name
        , SPLIT(CustomerName, ' ')[1] AS last_name
        , EmailAddress
    FROM tv_customer_AW;
    """
    df = spark.sql(query)


In [0]:
if entityName == 'products_AW':
    query =  """
    SELECT 
      ProductID
    , ProductName
    , Category
    , ListPrice
    from tv_products_AW;
    """
    df = spark.sql(query)


In [0]:
%sql
SELECT * FROM tv_sales_order_AW

In [0]:
if entityName == 'sales_order_AW':
    query =  """SELECT
      tso.SalesOrderNumber AS order_id
    , tso.OrderDate AS order_date
    , tca.customer_id as customer_id
    , tpa.ProductID AS product_id
    , tpa.ListPrice AS product_price
    , tso.UnitPrice AS unit_price
    , tso.TaxAmount as tax_amount
    , tso.UnitPrice + tso.TaxAmount AS total_sale 

    FROM tv_sales_order_AW tso
    JOIN tv_customer_AW tca ON tso.EmailAddress = tca.EmailAddress
    JOIN tv_products_AW tpa ON tso.Item = tpa.ProductName AND tso.UnitPrice = tpa.ListPrice
    """
    df = spark.sql(query)


In [0]:
# Check data exists:
if df.count() > 0:
    # Set location for delta table:
    file_path = f"/{sourceSystemName}/{entityName}"
    
    # Write to delta table:
    df.write.mode('overwrite').format('delta').option('header', 'true').save(GOLD + file_path)

    # Output file_path:
    dbutils.notebook.exit(file_path)